In [ ]:
%pip install datasets
%pip install transformers
%pip install huggingface_hub

  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.7/32.7 MB 15.5 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached aiosignal-1.4.0-py3-none-any.whl (7.5 kB)
Using cached async_timeout-5.0.1-py3-none-any.whl (6.2 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from datasets import load_dataset

ds = load_dataset("allenai/WildChat-1M")